## Imports

In [102]:
import os
import wave
import numpy as np

## Deleteing unneeded folders

In [2]:
mainFolderLocation = "ABI-1 Corpus/accents/"
listOfAccents = os.listdir(mainFolderLocation)

# Goes through list of accent folders
for accent in listOfAccents:
    accentFolderLocation = mainFolderLocation + accent + "/"
    if accent == ".DS_Store":
        os.remove(accentFolderLocation)

    accentContent = os.listdir(accentFolderLocation)

    # Goes through list of gender folders
    for genders in accentContent:
        genderFolderLocation = accentFolderLocation + genders + "/"
        if genders == ".DS_Store":
          os.remove(genderFolderLocation)

        genderContent = os.listdir(genderFolderLocation)
        
        # Goes through list of participant folders
        for participant in genderContent:
            participantFolderLocation = genderFolderLocation + participant + "/"
            if participant == ".DS_Store":
                os.remove(participantFolderLocation)

            participantContent = os.listdir(participantFolderLocation)

            for file in participantContent:
                fileAddress = participantFolderLocation + file
                if file[-3:] != "wav":
                    os.remove(fileAddress)

                if file[:12] != "shortpassage":
                    os.remove(fileAddress)


## Obtaining List of directories

In [3]:
def getSpeakerRoots(datapath = "ABI-1 Corpus\\accents"):
    speakerList = []
    accentSubfolder = [f.path for f in os.scandir(datapath) if f.is_dir()]
    for accent in accentSubfolder:
        for gender in ["female", "male"]:
            speakerFolders = os.listdir(os.path.join(accent, gender))
            for speaker in speakerFolders:
                if not speaker.startswith("."):
                    speakerList.append(os.path.join(accent, gender, speaker))
        
    return speakerList

In [4]:
speakerList = getSpeakerRoots()

## Getting .wav files

In [5]:
def getWavFile(datapath):
    files = os.listdir(datapath)
    filesWav = [i for i in files if i.endswith(".wav")]
    return filesWav

In [6]:
speakers = []
for i in speakerList:
    speakers.append(i[-6:])

speakerFilePairs = {}
for i in range(len(speakers)):
    if speakers[i] in speakerFilePairs:
        speakerFilePairs[speakers[i] + "-1"] = [getWavFile(speakerList[i]), speakerList[i]]
    else:
        speakerFilePairs[speakers[i]] = [getWavFile(speakerList[i]), speakerList[i]]

## Segmenting the data

In [43]:
folderName = "SegmentedDataset"
if os.path.exists(folderName) == False:
    os.mkdir(folderName)

for speaker in list(speakerFilePairs.keys()):
    speakerFolder = folderName + "\\" + speaker
    if os.path.exists(speakerFolder) == False:
        os.mkdir(speakerFolder)

In [136]:
def splitWavFile(inputFile, seconds):
    fileLocation = inputFile[1] + "\\" + inputFile[0][0]
    with wave.open(fileLocation, "rb") as wavFile:
        params = wavFile.getparams()
        sampleRate = params.framerate
        totalFrames = params.nframes

        # Calculate the number of frames per segment
        framesPerSegment = int(seconds * sampleRate)
        totalSegment = totalFrames // framesPerSegment
        
        for segmentIndex in range(totalSegment):
            startFrame = segmentIndex * framesPerSegment
            endFrame = (segmentIndex+1) * framesPerSegment

            wavFile.setpos(startFrame)
            audioData = np.frombuffer(wavFile.readframes(framesPerSegment), dtype=np.int16)
            outputFile = "SegmentedDataset\\" + inputFile[1][-6:] + "\\chunk" + str(segmentIndex) + ".wav"

            with wave.open(outputFile, "wb") as outputWav:
                outputWav.setparams(params)
                outputWav.writeframes(audioData.tobytes())

In [139]:
indexes = list(speakerFilePairs.keys())
for index in indexes:
    splitWavFile(speakerFilePairs[index], 3)